# Тонкая настройка модели GPT-2

Этот ноутбук демонстрирует процесс тонкой настройки модели GPT-2 на пользовательском наборе данных в формате JSON Lines. Включает установку зависимостей, загрузку данных, токенизацию, обучение модели и сохранение результатов.

In [1]:
# Установка необходимых библиотек
!pip install "transformers>=4.41.0"
!pip install datasets torch>=2.0.0 tensorboard

In [2]:
import os
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import load_dataset

if not os.path.exists("data"):
    os.makedirs("data")
with open("data/my_corpus.jsonl", "w", encoding="utf-8") as f:
    f.write('{"text": "Пример текста для обучения модели."}\n')
    f.write('{"text": "Еще одна строка текста для демонстрации."}\n')
    f.write('{"text": "Тонкая настройка GPT-2 на пользовательских данных."}\n')

In [3]:
import json
from datasets import Dataset, DatasetDict

# Читаем данные из файла вручную
data = []
with open("data/my_corpus.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line))

# Создаем Dataset из списка словарей
raw_dataset = Dataset.from_dict({"text": [item["text"] for item in data]})

# Создаем DatasetDict с разделом 'train'
dataset = DatasetDict({
    'train': raw_dataset
})

print("Данные успешно загружены вручную.")
print(dataset)

Данные успешно загружены вручную.
DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 3
    })
})


In [4]:
# Инициализация токенизатора и модели
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Установка pad_token равным eos_token, чтобы избежать предупреждений
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [5]:
# Функция токенизации
def tokenize(example):
    tokenized = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# Токенизация набора данных
tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=['text'])

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [6]:
# Определение аргументов для обучения
training_args = TrainingArguments(
    output_dir="./model",
    logging_dir="./logs",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    save_steps=500,
    logging_steps=100,
    save_total_limit=2,
    logging_strategy="steps",
    save_strategy="steps",
    eval_strategy="no",
    load_best_model_at_end=False,
    report_to="tensorboard"
)

In [7]:
# Инициализация Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
)

In [8]:
# Начало обучения
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=6, training_loss=1.579252560933431, metrics={'train_runtime': 147.9049, 'train_samples_per_second': 0.061, 'train_steps_per_second': 0.041, 'total_flos': 2351628288000.0, 'train_loss': 1.579252560933431, 'epoch': 3.0})

In [9]:
# Сохранение финальной модели и токенизатора
model.save_pretrained("./model/final")
tokenizer.save_pretrained("./model/final")

print("Обучение завершено. Модель и токенизатор сохранены в ./model/final")

Обучение завершено. Модель и токенизатор сохранены в ./model/final
